<a href="https://colab.research.google.com/github/NNL-Keerthana/DeepLearning_EEG/blob/main/ATCNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
dataset_path = '/content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0'
os.chdir(dataset_path)
#/content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/subject-info.csv

In [4]:
!pip install mne

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 19.4 MB/s eta 0:00:00


In [5]:
import mne
import numpy as np
import pandas as pd

# Load subject info from subject-info.csv
subject_info = pd.read_csv('/content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/subject-info.csv')

# Function to calculate band-wise PSD
def calculate_band_psd(raw, sfreq, freq_bands):
    psds, freqs = mne.time_frequency.psd_array_multitaper(raw.get_data(), sfreq=sfreq, fmin=0.5, fmax=40, adaptive=True, normalization='full', verbose=0)
    band_psd = dict()
    for band, (fmin, fmax) in freq_bands.items():
        band_psd[band] = np.mean(psds[:, (freqs >= fmin) & (freqs < fmax)], axis=1)
    return band_psd

# Define frequency bands of interest
freq_bands = {
    'Delta': (1, 4),
    'Theta': (4, 8),
    'Alpha': (8, 12),
    'Beta': (12, 30),
    'Gamma': (30, 40)
}

# Function to load and preprocess data
def load_and_save_eeg_data(subject_info, freq_bands):
    X_list = []
    y_list = []
    for idx, row in subject_info.iterrows():
        subject_id = row['Subject']  # Directly use 'Subject' column from your CSV
        # File paths
        file_rest = f'/content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/{subject_id}_1.edf'
        file_task = f'/content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/{subject_id}_2.edf'

        # Load EDF files
        raw_rest = mne.io.read_raw_edf(file_rest, preload=True)
        raw_task = mne.io.read_raw_edf(file_task, preload=True)

        # Filter raw data
        raw_rest.filter(1, 40, fir_design='firwin')
        raw_task.filter(1, 40, fir_design='firwin')

        # Calculate PSD
        psd_rest = calculate_band_psd(raw_rest, raw_rest.info['sfreq'], freq_bands)
        psd_task = calculate_band_psd(raw_task, raw_task.info['sfreq'], freq_bands)

        # Flatten PSD values for machine learning input
        X_rest = np.concatenate([psd_rest[band] for band in freq_bands.keys()])
        X_task = np.concatenate([psd_task[band] for band in freq_bands.keys()])

        # Labels: 0 for rest, 1 for task
        y_rest = np.zeros(X_rest.shape[0])
        y_task = np.ones(X_task.shape[0])

        # Concatenate features and labels
        X_list.append(X_rest)
        X_list.append(X_task)
        y_list.append(y_rest)
        y_list.append(y_task)

    X = np.array(X_list)
    y = np.array(y_list)
    np.save('X_psd.npy', X)
    np.save('y_psd.npy', y)
    print(f"Saved PSD features to 'X_psd.npy' and labels to 'y_psd.npy'")

# Run the function to calculate and save PSD features
load_and_save_eeg_data(subject_info, freq_bands)


Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/Subject00_1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 90999  =      0.000 ...   181.998 secs...
Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/Subject00_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 ...    61.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 H

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/Subject01_1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 90999  =      0.000 ...   181.998 secs...
Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/Subject01_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 ...    61.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 H

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/Subject02_1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 90999  =      0.000 ...   181.998 secs...
Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/Subject02_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 ...    61.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 H

...

Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/Subject34_1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 90999  =      0.000 ...   181.998 secs...
Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/Subject34_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 ...    61.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 H

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/Subject35_1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 90999  =      0.000 ...   181.998 secs...
Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/Subject35_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 ...    61.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 H

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Saved PSD features to 'X_psd.npy' and labels to 'y_psd.npy'


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, DepthwiseConv2D, AveragePooling2D, Flatten, Dense, Dropout, Activation
from tensorflow.keras.constraints import max_norm

# Load saved PSD features and labels
X = np.load('X_psd.npy')
y = np.load('y_psd.npy')

# Ensure the data is reshaped appropriately for EEGNet
print(f"Original X shape: {X.shape}")
print(f"Original y shape: {y.shape}")

# EEGNet expects 4D input: (samples, 1, channels, 1)
X = X.reshape(X.shape[0], 1, X.shape[1], 1)
print(f"Reshaped X shape: {X.shape}")

# Ensure y is a 1D array before one-hot encoding
if y.ndim > 1:
    y = y[:, 0]
print(f"Corrected y shape: {y.shape}")

# One-hot encode labels (this part remains the same)
y = to_categorical(y)
print(f"One-hot encoded y shape: {y.shape}")

# Flatten X for the train_test_split
X_flat = X.reshape(X.shape[0], -1)
print(f"Flattened X shape: {X_flat.shape}")

# Use argmax to get 1D array of labels for stratification
y_for_stratify = np.argmax(y, axis=1)  # Get the index of the class with highest probability

# Split into train and test sets (use y_for_stratify for stratification)
X_train_flat, X_test_flat, y_train, y_test = train_test_split(
    X_flat, y, test_size=0.2, random_state=42, stratify=y_for_stratify
)

# Reshape back to original dimensions after the split
X_train = X_train_flat.reshape(X_train_flat.shape[0], 1, -1, 1)
X_test = X_test_flat.reshape(X_test_flat.shape[0], 1, -1, 1)

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

# Normalize the data
scaler = StandardScaler()

# Flatten the training data, fit the scaler, and reshape back to the original shape
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_train_scaled = scaler.fit_transform(X_train_flat)
X_train = X_train_scaled.reshape(X_train.shape)

# Flatten the testing data, apply the scaler, and reshape back to the original shape
X_test_flat = X_test.reshape(X_test.shape[0], -1)
X_test_scaled = scaler.transform(X_test_flat)
X_test = X_test_scaled.reshape(X_test.shape)

print(f"Normalized X_train shape: {X_train.shape}")
print(f"Normalized X_test shape: {X_test.shape}")


Original X shape: (72, 105)
Original y shape: (72, 105)
Reshaped X shape: (72, 1, 105, 1)
Corrected y shape: (72,)
One-hot encoded y shape: (72, 2)
Flattened X shape: (72, 105)
X_train shape: (57, 1, 105, 1)
X_test shape: (15, 1, 105, 1)
Normalized X_train shape: (57, 1, 105, 1)
Normalized X_test shape: (15, 1, 105, 1)




## ATCNet

References (Official Docs): https://github.com/Altaheri/EEG-ATCNet



Preprocessing specific to ATCNet Architecture

In [7]:
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load saved PSD features and labels
X = np.load('X_psd.npy')
y = np.load('y_psd.npy')

# Ensure the data is reshaped appropriately for the model
X = X.reshape(X.shape[0], X.shape[1], 1, 1)  # Reshape to (samples, channels, height, width)

# Ensure y is a 1D array
if y.ndim > 1:
    y = y[:, 0]

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [8]:
# ACTNet Model architecture
def build_atcnet(input_shape):
    model = Sequential([
        Conv2D(16, (1, 64), padding='same', input_shape=input_shape),
        BatchNormalization(),
        Activation('relu'),
        AveragePooling2D(pool_size=(1, 4)),
        Conv2D(32, (1, 32), padding='same'),
        BatchNormalization(),
        Activation('relu'),
        AveragePooling2D(pool_size=(1, 4)),
        Flatten(),
        Dense(64, kernel_constraint=max_norm(2.)),
        BatchNormalization(),
        Activation('relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')  # Output a single probability score
    ])
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, AveragePooling2D, Flatten, Dense, Dropout
from tensorflow.keras.constraints import max_norm

def build_atcnet(input_shape):
    model = Sequential([
        # Convolutional Block 1
        Conv2D(16, (1, 64), padding='same', input_shape=input_shape),
        BatchNormalization(),
        Activation('relu'),
        AveragePooling2D(pool_size=(1, 1)),
        Dropout(0.5),  # Dropout added after convolutional layer

        # Convolutional Block 2
        Conv2D(32, (1, 32), padding='same'),
        BatchNormalization(),
        Activation('relu'),
        AveragePooling2D(pool_size=(1, 1)),
        Dropout(0.5),  # Dropout added after convolutional layer

        # Flatten and Dense layers
        Flatten(),
        Dense(64, kernel_constraint=max_norm(2.)),
        BatchNormalization(),
        Activation('relu'),
        Dropout(0.5),  # Dropout added before final dense layer

        # Output layer
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

# Initialize and build the model
model = build_atcnet(input_shape=X_train.shape[1:])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 105, 1, 16)        1040      
                                                                 
 batch_normalization (Batch  (None, 105, 1, 16)        64        
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 105, 1, 16)        0         
                                                                 
 average_pooling2d (Average  (None, 105, 1, 16)        0         
 Pooling2D)                                                      
                                                                 
 dropout (Dropout)           (None, 105, 1, 16)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 105, 1, 32)        1

In [10]:
from tensorflow.keras.optimizers import Adam

# Adjust learning rate and optimizer settings
learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)

# Compile the model with updated optimizer
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Predict probabilities for test data
y_pred_proba = model.predict(X_test)

# Convert probabilities to class labels
y_pred = (y_pred_proba > 0.5).astype(int)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-score: {f1:.4f}')

Epoch 1/50
2/2 [==============================] - 9s 676ms/step - loss: 0.6870 - accuracy: 0.5111 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 2/50
2/2 [==============================] - 0s 56ms/step - loss: 0.6954 - accuracy: 0.6000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/50
2/2 [==============================] - 0s 59ms/step - loss: 0.7120 - accuracy: 0.5333 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 4/50
2/2 [==============================] - 0s 55ms/step - loss: 0.7380 - accuracy: 0.5111 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 5/50
2/2 [==============================] - 0s 44ms/step - loss: 0.7447 - accuracy: 0.3778 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 6/50
2/2 [==============================] - 0s 68ms/step - loss: 0.7062 - accuracy: 0.5333 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 7/50
2/2 [==============================] - 0s 56ms/step - loss: 0.6884 - accuracy: 0.6222 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 8/50
2/2 [==========

Any further attempts at improving the Accuracy are met with 0 Precision and Recall which is a bad sign

Tried changing DropOut, Epochs and Learning Rate.

### Inference:

For medical diagnostics, high recall can be prioritized to avoid missing positive cases, even if it leads to more false positives. Thus I have made tweaks in architechture to support high Recall

Original ATCNET Docs: https://github.com/Altaheri/EEG-ATCNet

In [17]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, AveragePooling2D, Flatten, Dense, Dropout, Reshape, Permute, Multiply, Add, Input
from tensorflow.keras.constraints import max_norm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load saved PSD features and labels
X = np.load('X_psd.npy')
y = np.load('y_psd.npy')

# Ensure the data is reshaped appropriately for the model
X = X.reshape(X.shape[0], X.shape[1], 1, 1)  # Reshape to (samples, channels, height, width)

# Ensure y is a 1D array
if y.ndim > 1:
    y = y[:, 0]

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Define your ATCNet model architecture
def build_atcnet(input_shape):
    model = Sequential([
        Conv2D(16, (1, 64), padding='same', input_shape=input_shape),
        BatchNormalization(),
        Activation('relu'),
        AveragePooling2D(pool_size=(1, 1)),
        Conv2D(32, (1, 32), padding='same'),
        BatchNormalization(),
        Activation('relu'),
        AveragePooling2D(pool_size=(1, 1)),
        Flatten(),
        Dense(64, kernel_constraint=max_norm(2.)),
        BatchNormalization(),
        Activation('relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# Initialize and build the model
input_shape = X_train.shape[1:]
model = build_atcnet(input_shape)
# model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

# Evaluate the model
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)

accuracy = accuracy_score(y_test, y_pred_binary)
precision = precision_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)
f1 = f1_score(y_test, y_pred_binary)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")


Epoch 1/100
2/2 [==============================] - 4s 367ms/step - loss: 0.6933 - accuracy: 0.4222 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 2/100
2/2 [==============================] - 0s 61ms/step - loss: 0.6931 - accuracy: 0.5111 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 3/100
2/2 [==============================] - 0s 62ms/step - loss: 0.6930 - accuracy: 0.4889 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 4/100
2/2 [==============================] - 0s 63ms/step - loss: 0.6931 - accuracy: 0.4889 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 5/100
2/2 [==============================] - 0s 62ms/step - loss: 0.6934 - accuracy: 0.4667 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 6/100
2/2 [==============================] - 0s 45ms/step - loss: 0.6932 - accuracy: 0.4667 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 7/100
2/2 [==============================] - 0s 45ms/step - loss: 0.6931 - accuracy: 0.5333 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 8/100
2/2 [==